From from this article: [Time Series — ARIMA vs. SARIMA vs. LSTM: Hands-On Tutorial](https://towardsdatascience.com/time-series-arima-vs-sarima-vs-lstm-hands-on-tutorial-bd5630298da3)

First we want to start off with downloading the data. It can be had from [here](https://archive.ics.uci.edu/dataset/360/air+quality). It's around 1 MB.

In [1]:
# import libraries
import pandas as pd
import numpy as np

from ucimlrepo import fetch_ucirepo

# Fetch dataset
air_quality = fetch_ucirepo(id=360)

Get the data from the data set.

Same thing with many names:
- Features, X, input variables, independent variables
- Targets, y, output variables, dependent variables

In [ ]:

# Data (as pandas dataframes)
air_quality_df = air_quality.data.features
air_quality_df.head()